# Introduction

This notebook documents test suites in Checklist. If you are not already familiar with creating tests in Checklist, consider reading the MFT Examples notebook.

## Setup
First, let's import the libraries and load the model.

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import random
import json
import checklist
from checklist.editor import Editor
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_types import MFT
from checklist.test_suite import TestSuite
from torch.nn import functional as F
from typing import List
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initialize random seed
# Remove this code to experiment with random samples
random.seed(123)
torch.manual_seed(456)

In [3]:
# Load pretrained model tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
device = 'cuda'
model.eval()
model.to(device)
"Model loaded"

'Model loaded'

# Creating a Test Suite

Checklist can run multiple tests in a test suite. Tests can be grouped by capability and results can be explored in a visual table.

We will create a test suite called 'Same Token Prediction' with 3 MFTs. Each MFT will test if the token substituted into the prompt template also appears in the generated text.

For example, if we prompt the model with "The **dog** is running in the zoo" and the model responds with "The **dog** looks very happy", then it passes the test because the same animal appears in the model's response.

## Creating the MFTs
### MFT 1: Same animal appears in response
This MFT uses an `{animal}` placeholder in the template. The expectation function checks that the same animal appears in the prediction.

In [4]:
editor = Editor()
animal_prompts = editor.template("The {animal} is running in the zoo", animal=["dog", "cat", "giraffe", "aardvark"], meta=True)
animal_prompts.data

['The dog is running in the zoo',
 'The cat is running in the zoo',
 'The giraffe is running in the zoo',
 'The aardvark is running in the zoo']

In [5]:
def contains_same_animal(x, pred, conf, label=None, meta=None):
    return meta['animal'] in pred

In [6]:
same_animal_expect_fn = Expect.single(contains_same_animal)
same_animal_test = MFT(**animal_prompts, name='Same animal in response', description='The response contains the same animal mentioned in the prompt.', expect=same_animal_expect_fn)

### MFT 2: Same country appears in response
This MFT uses a `{country}` placeholder in the template. The expectation function checks that the same country appears in the prediction.

In [7]:
country_prompts = editor.template("Earlier today, scientists from {country} discovered  ", meta=True, nsamples=10)
country_prompts

MunchWithAdd({'meta': [{'country': 'Algeria'}, {'country': 'Nigeria'}, {'country': 'Trinidad and Tobago'}, {'country': 'Chile'}, {'country': 'Tuvalu'}, {'country': 'Guinea'}, {'country': 'Djibouti'}, {'country': 'Poland'}, {'country': 'Syria'}, {'country': 'Ghana'}], 'data': ['Earlier today, scientists from Algeria discovered  ', 'Earlier today, scientists from Nigeria discovered  ', 'Earlier today, scientists from Trinidad and Tobago discovered  ', 'Earlier today, scientists from Chile discovered  ', 'Earlier today, scientists from Tuvalu discovered  ', 'Earlier today, scientists from Guinea discovered  ', 'Earlier today, scientists from Djibouti discovered  ', 'Earlier today, scientists from Poland discovered  ', 'Earlier today, scientists from Syria discovered  ', 'Earlier today, scientists from Ghana discovered  ']})

In [8]:
def contains_same_country(x, pred, conf, label=None, meta=None):
    return meta['country'] in pred

In [9]:
same_country_expect_fn = Expect.single(contains_same_country)
same_country_test = MFT(**country_prompts, name='Same country in response', description='The response contains the same country mentioned in the prompt.', expect=same_country_expect_fn)

### MFT 3: Same person appears in response
This MFT uses a `{first_name}` placeholder in the template. The expectation function checks that the same first name appears in the prediction.

In [10]:
person_prompts = editor.template("{first_name} is my neighbor.", meta=True, nsamples=10)
person_prompts

MunchWithAdd({'meta': [{'first_name': 'Jonathan'}, {'first_name': 'Larry'}, {'first_name': 'Philip'}, {'first_name': 'Mike'}, {'first_name': 'Suzanne'}, {'first_name': 'Roy'}, {'first_name': 'Jason'}, {'first_name': 'Grace'}, {'first_name': 'Kathleen'}, {'first_name': 'Melissa'}], 'data': ['Jonathan is my neighbor.', 'Larry is my neighbor.', 'Philip is my neighbor.', 'Mike is my neighbor.', 'Suzanne is my neighbor.', 'Roy is my neighbor.', 'Jason is my neighbor.', 'Grace is my neighbor.', 'Kathleen is my neighbor.', 'Melissa is my neighbor.']})

In [11]:
def contains_same_person(x, pred, conf, label=None, meta=None):
    return meta['first_name'] in pred

In [12]:
same_person_expect_fn = Expect.single(contains_same_person)
same_person_test = MFT(**person_prompts, name='Same person in response', description='The response contains the same person\'s first name mentioned in the prompt.', expect=same_person_expect_fn)

## Adding the tests to the suite
The `TestSuite()` constructor creates an empty test suite. Tests can be added one by one using `suite.add(test)`. The optional `capability` parameter can be used to label and group tests that test similar capabilities.

In [13]:
suite = TestSuite()

In [14]:
suite.add(same_animal_test, capability="Same Token Prediction")
suite.add(same_country_test, capability="Same Token Prediction")
suite.add(same_person_test, capability="Same Token Prediction")

## Generating the predictions
Now we define the function that Checklist will use to generate predictions from the model. The predictions need to be returned in the form `([predictions], [scores])`, so we will wrap the `generate_sentences()` function with `PredictorWrapper.wrap_predict()` to automatically create a tuple `([predictions], [1, 1, ...])`

In [15]:
def generate_sentence(prompt: str) -> str:
    token_tensor = tokenizer.encode(prompt, return_tensors='pt').to(device) # return_tensors = "pt" returns a PyTorch tensor
    out = model.generate(
        token_tensor,
        do_sample=True,
        min_length=10,
        max_length=50,
        num_beams=1,
        temperature=1.0,
        no_repeat_ngram_size=2,
        early_stopping=False,
        output_scores=True,
        return_dict_in_generate=True)
    text = tokenizer.decode(out.sequences[0], skip_special_tokens=True)
    return text[len(prompt):]

In [16]:
def generate_sentences(prompts: List[str]) -> List[str]:
    sentences = []
    for prompt in prompts:
        sentences.append(generate_sentence(prompt))
    return sentences

In [17]:
wrapped_generator = PredictorWrapper.wrap_predict(generate_sentences)
wrapped_generator(["Hello, nice to meet you.", "Goodbye, see you later."])

([' Now, let us begin to talk…\n\nI heard you guys were coming.\n: I thought you were from there, right?\n (Slight sigh, looks back at Vixen) Yeah,',
  ' Goodbye, goodbye, dear." But when I heard that I felt almost lost.\n\nMy wife gave me the same answer of "Thank you."\n.'],
 array([1., 1.]))

## Running the suite
We can now run the suite and view the results.

In [18]:
suite.run(wrapped_generator, overwrite=True)

Running Same animal in response
Predicting 4 examples
Running Same country in response
Predicting 10 examples
Running Same person in response
Predicting 10 examples


In [19]:
def format_example(x, pred, conf, label=None, meta=None): 
    return 'Prompt:      %s\nCompletion:      %s' % (x, pred) 

In [20]:
suite.summary(format_example_fn = format_example)

Same Token Prediction

Same animal in response
Test cases:      4
Fails (rate):    4 (100.0%)

Example fails:
Prompt:      The giraffe is running in the zoo
Completion:       when he runs out of time

'I was just looking at the wall in 'You're Coming Down' and it said: "You need to get down and run down to the river, then if
----
Prompt:      The cat is running in the zoo
Completion:      , but I'll only bring back a one year old. There's one guy at the park that is crazy. He was doing some nice things but it's definitely going to end up being one of those stories that
----
Prompt:      The dog is running in the zoo
Completion:      , she says, and a friend of her walks in on them, shouting things like, "I was scared, can you please sit down"?

He's also apparently had a seizure and an eye scratch,
----


Same country in response
Test cases:      10
Fails (rate):    10 (100.0%)

Example fails:
Prompt:      Earlier today, scientists from Djibouti discovered  
Completion:       two new s

In [21]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Same animal in respo…

# Using files with test suites

Some models cannot be run directly on the same machine that is running the Checklist test suite. For instance, a model might need to run in a specially configured lab environment. In this case, Checklist does not have to receive the predictions from the model directly. The predictions can be saved to a file, then the test suite can check the predictions from the file.

## Exporting a test suite to a file
First, let's create a file that contains all the prompts that we will send to the model.

### Accessing test suite data internally
Tests are stored in `suite.tests`, which is a dictionary mapping the test name to the test.

In [22]:
for key in suite.tests.keys():
    print(key)

Same animal in response
Same country in response
Same person in response


We can access the test information by like this:

In [23]:
suite.tests['Same animal in response'].data

['The dog is running in the zoo',
 'The cat is running in the zoo',
 'The giraffe is running in the zoo',
 'The aardvark is running in the zoo']

In [24]:
suite.tests['Same animal in response'].meta

[{'animal': 'dog'},
 {'animal': 'cat'},
 {'animal': 'giraffe'},
 {'animal': 'aardvark'}]

### Exporting to JSON file with to_raw_file()
TestSuite's `to_raw_file()` function exports a test suite to a file. The `format_fn` parameter allows us to control how each example in the suite is printed to the file. We can use `format_fn` to print the examples in a JSON format.

In [25]:
def suite_to_json_file(suite, filename):
    class Counter:
        def __init__(self):
            self.count = 0
        def get_count(self):
            self.count += 1
            return self.count
    
    counter = Counter()
    total_tests = 0
    for t in suite.tests.values():
        total_tests += len(t.data)
        
    def json_format_fn(x):
        example_id = counter.get_count()
        json_str = ""
        if example_id == 1:
            json_str = '{"examples": ['
        json_str += json.dumps({'content': x, 'id': example_id}) + ","
        if example_id == total_tests:
            # remove trailing comma
            json_str = json_str[:len(json_str)-1]
            json_str += "]}"
        return json_str
    
    suite.to_raw_file(filename, format_fn = json_format_fn)

In [26]:
suite_to_json_file(suite, 'same_token_suite.json')

In [27]:
cat 'same_token_suite.json'

{"examples": [{"content": "The dog is running in the zoo", "id": 1},
{"content": "The cat is running in the zoo", "id": 2},
{"content": "The giraffe is running in the zoo", "id": 3},
{"content": "The aardvark is running in the zoo", "id": 4},
{"content": "Earlier today, scientists from Algeria discovered  ", "id": 5},
{"content": "Earlier today, scientists from Nigeria discovered  ", "id": 6},
{"content": "Earlier today, scientists from Trinidad and Tobago discovered  ", "id": 7},
{"content": "Earlier today, scientists from Chile discovered  ", "id": 8},
{"content": "Earlier today, scientists from Tuvalu discovered  ", "id": 9},
{"content": "Earlier today, scientists from Guinea discovered  ", "id": 10},
{"content": "Earlier today, scientists from Djibouti discovered  ", "id": 11},
{"content": "Earlier today, scientists from Poland discovered  ", "id": 12},
{"content": "Earlier today, scientists from Syria discovered  ", "id": 13},
{"content": "Earlier today, scientists from Ghana disc

## Importing the test suite JSON
The JSON file we created can be imported back into a Python object by using `json.load()`.

In [28]:
import json
f = open('same_token_suite.json', 'r')
suite_dict = json.load(f)
f.close()
suite_dict['examples'][0:3]

[{'content': 'The dog is running in the zoo', 'id': 1},
 {'content': 'The cat is running in the zoo', 'id': 2},
 {'content': 'The giraffe is running in the zoo', 'id': 3}]

## Generating predictions from the loaded data
Our data has been loaded into a variable named `suite_dict`. Now we can read each example from `suite_dict` and generate the predictions. Each prediction will be written to another file named `same_token_suite_predictions.json`, which will be sent to Checklist to evaluate the results.

In [29]:
with open('same_token_suite_predictions.json', 'w') as f:
    for example in suite_dict['examples']:
        prediction = generate_sentence(example['content'])
        prediction = prediction.replace('"', '\"')
        f.write(json.dumps({'prediction': prediction, 'id': example['id']}) + '\n')

In [30]:
cat 'same_token_suite_predictions.json'

{"prediction": ", too, but the vet said she only had a couple hours to live with her.", "id": 1}
{"prediction": ", but I thought this was cute (he's definitely a feline with his eyes!). I want my cat to grow up and be happy; to be successful and healthy.", "id": 2}
{"prediction": ", after being kept in a girder and fed by others. He's having a bad fit when he makes contact with an adult giraffeater.\n\nSome girasses were so scared they had to", "id": 3}
{"prediction": " to avoid being caught up in a battle at Bauberwald.\n\nThe zoo was set up yesterday to try to prevent a possible resurgence of a wild animal that is already in conflict with Berlin", "id": 4}
{"prediction": "\u00c2 a massive hole in Earth's crust, allowing an extremely high-resolution search for the source of the new finding.\n\nAn image from images from the European Space Agency's Gaia satellite reveals a new", "id": 5}
{"prediction": "\ue001 \ue601\u00a0 the first-ever study of the intercellular boundary between human

## Reading test results from file
TestSuite has a `run_from_file()` function that reads the predictions line by line from a file. The `format_fn` parameter is used to parse each line of the file. Our format function, `read_json_prediction()`, converts the JSON object into the predicted text.

In [31]:
def read_json_prediction(x):
    test_output = json.load(x)
    return test_output['prediction']
suite.run_from_file('same_token_suite_predictions.json', file_format='pred_only', format_fn = read_json_prediction, overwrite=True)

In [32]:
suite.summary(format_example_fn = format_example)

Same Token Prediction

Same animal in response
Test cases:      4
Fails (rate):    2 (50.0%)

Example fails:
Prompt:      The dog is running in the zoo
Completion:      {"prediction": ", too, but the vet said she only had a couple hours to live with her.", "id": 1}
----
Prompt:      The aardvark is running in the zoo
Completion:      {"prediction": " to avoid being caught up in a battle at Bauberwald.\n\nThe zoo was set up yesterday to try to prevent a possible resurgence of a wild animal that is already in conflict with Berlin", "id": 4}
----


Same country in response
Test cases:      10
Fails (rate):    10 (100.0%)

Example fails:
Prompt:      Earlier today, scientists from Ghana discovered  
Completion:      {"prediction": "_____________________________________ _______ with their discovery of a new type of chiral, a key element required to prevent the formation of an electron in the ribosome. The discovery will also help in understanding what makes", "id": 14}
----
Prompt:      Ear

In [33]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Same animal in respo…